In [120]:
import sqlite3
import datetime as dt
import pandas as pd
import re

In [92]:
reset = False

In [90]:
# set up the database
conn = sqlite3.connect('ChoreTracker.sqlite3')
cursor = conn.cursor()
tables = [t[0] for t in cursor.execute(""" SELECT name FROM sqlite_schema WHERE type='table' ORDER BY name;  """).fetchall()]

In [91]:
tables

['choreinstances',
 'chorerates',
 'choreresponsibilities',
 'chores',
 'credentials',
 'people',
 'roles',
 'sqlite_sequence']

In [12]:
# Setting up the data model

r = cursor.execute("""
create table if not exists chores (
    ChoreID integer primary key AUTOINCREMENT,
    name text, 
    schedule text, -- how frequently does this chore happen (in days)?
    start_date text, -- when is the first day that this chore applies?
    start_time text, -- what time of day does the chore window open?
    window text, -- how many hours before the chore needs to be finished? 
    repeats INTEGER -- boolean to say whether this chore sould repeat or not
    );
""")


In [136]:
class NoChoreNameSuppliedError(Exception):
    pass

class NoStartDateSuppliedError(Exception):
    pass

class NoStartTimeSuppliedError(Exception):
    pass

class IncorrectTimeWindowFormatError(Exception):
    pass

def create_chore(
    name: str = None,
    schedule: str = '1D',
    start_date: dt.date = dt.date.today(),
    start_time: dt.time = dt.time(7),
    window: str = '4H',
    repeats: bool = True
):
    if name is None:
        raise NoChoreNameSuppliedError
    if start_date is None:
        raise NoStartDateSuppliedError
    if start_time is None:
        raise NoStartTimeSuppliedError
    
    if not re.match(r'\d+[YWDMH]', schedule):
        raise IncorrectTimeWindowFormatError

    if not re.match(r'\d+[YWDMH]', window):
        raise IncorrectTimeWindowFormatError

    update_db(f"""
        INSERT INTO chores 
        (name, schedule, start_date, start_time, window, repeats)
        VALUES
        ('{name}', '{schedule}', '{start_date}', '{start_time}', '{window}', {repeats})
        """)
    
    return True

def get_chores():
    return query_db(q="""
        SELECT * FROM chores
    """)

In [141]:
create_chore(name='Empty dishwasher (bottom)', schedule='2D', start_date=dt.date.today())

True

In [142]:
get_chores()

(['ChoreID',
  'name',
  'schedule',
  'start_date',
  'start_time',
  'window',
  'repeats'],
 [(1, 'Empty dishwasher (top)', '2D', '2023-01-19', '07:00:00', '4H', 1),
  (2, 'Empty dishwasher (top)', '2D', '2023-01-20', '07:00:00', '4H', 1),
  (3, 'Empty dishwasher (bottom)', '2D', '2023-01-20', '07:00:00', '4H', 1),
  (4, 'Empty dishwasher (bottom)', '2D', '2023-01-19', '07:00:00', '4H', 1)])

In [38]:
if reset:
    r = cursor.execute("""
    drop table roles;
    """)
    
r = cursor.execute("""
create table if not exists roles (
    RoleID integer primary key AUTOINCREMENT,
    RoleName text UNIQUE -- parent or child
    );
""")

if reset:
    for role in ['Parent', 'Child']:
        r = cursor.execute(f"""
            INSERT INTO roles (RoleName) VALUES ('{role}');
        """)

conn.commit()

In [51]:
if reset:
    r = cursor.execute(""" drop table people """)

In [49]:
# Keep track of individuals so chores can be assigned to them
r = cursor.execute("""
create table if not exists people (
    PersonID integer primary key AUTOINCREMENT,
    PersonName text, 
    RoleID INTEGER, -- whether person is parent or child
    CurrentBalance INTEGER -- balance in pence, display number/100 for proper currency
    );
""")


In [21]:

# We're going to have a daily scheduled process to calculate the day's chores, and pop them in this table
# so we can track whether repeated chores are carried out 
r = cursor.execute("""
create table if not exists choreinstances (
    ChoreInstanceID integer primary key,
    ChoreID INTEGER, 
    Completed INTEGER, -- Boolean
    CompletedBy INTEGER, -- this will store PersonID of whoever completed it
    Banked INTEGER -- has it been paid into the bank account?
    );
""")


In [22]:

# Who is responsible for each chore?
# More than one person can be responsible, so we have to be 
# able to track whether a multi-person chore has been completed and by whom
r = cursor.execute("""
create table if not exists choreresponsibilities (
    ResponsibilityID integer primary key,
    PersonID INTEGER, 
    ChoreID INTEGER
    );
""")


In [102]:
# What's the going rate for a chore these days?
r = cursor.execute("""
create table if not exists chorerates (
    RateID integer primary key,
    ChoreRate INTEGER, -- amount in pence
    StartDate TEXT -- when did the rate take effect?
    );
""")

In [112]:
cursor.execute(""" SELECT * FROM chorerates""").fetchall(), \
[c[0] for c in cursor.description]

([(1, 25, '2023-01-01'), (2, 25, '2023-01-01 19:39:00')],
 ['RateID', 'Rate', 'StartDate'])

In [116]:
def update_chore_rate(ChoreRate: int = 25):
    update_db(f""" 
        INSERT INTO chorerates 
        (Rate, StartDate) 
        VALUES ({ChoreRate}, '{dt.datetime.strftime(dt.datetime.now(), "%Y-%m-%d %H:%M:%S")}')
        """)

def get_chore_rates():
    return query_db(""" SELECT * FROM chorerates""")

In [119]:
get_chore_rates()

(['RateID', 'Rate', 'StartDate'],
 [(1, 25, '2023-01-01'),
  (2, 25, '2023-01-01 19:39:00'),
  (3, 30, '2023-01-19 20:07:35'),
  (4, 25, '2023-01-19 20:07:59')])

In [118]:
update_chore_rate(ChoreRate=25)

In [66]:

# Need to have some way of storing the parent passcode
# and potentially change it.

r = cursor.execute("""
create table if not exists credentials (
    CredentialID integer primary key,
    RoleID INTEGER, -- need one or both of these two
    PersonID INTEGER, -- need one or both of these two
    Passcode TEXT -- this should possibly be encrypted...
    );
""")

In [81]:

class UnrecognisedRoleTypeError(Exception):
    pass

def update_db(
    q: str = None,
    db_name: str = 'ChoreTracker.sqlite3',
    commit: bool = True
    ):
        conn = sqlite3.connect(db_name) 
        cursor = conn.cursor()
        r = cursor.execute(q)
        if commit:
            conn.commit()
        conn.close()
        return r

def query_db(
    q: str = None,
    db_name: str = 'ChoreTracker.sqlite3',
    ):
        conn = sqlite3.connect(db_name) 
        cursor = conn.cursor()
        r = cursor.execute(q).fetchall()
        cols = [c[0] for c in cursor.description]
        conn.close()
        return cols, r

def create_person(PersonName: str = None, RoleType: str = None, CurrentBalance: int = 0):
    if not RoleType in role_ids:
        raise UnrecognisedRoleTypeError

    r = update_db(q=f"""
        INSERT INTO people (PersonName, RoleID, CurrentBalance) VALUES ('{PersonName}', {role_ids[RoleType]}, {CurrentBalance});
    """, commit=True)




In [94]:
role_cols, roles = query_db(""" select * from roles """) # .fetchall()
roles
role_ids = dict([(dict(roles)[k], k) for k in dict(roles)])
role_ids

{'Parent': 2, 'Child': 3}

In [78]:
create_person(PersonName='Michelle', RoleType='Parent')

In [86]:
cols, people = query_db(q=""" select * from people """) # .fetchall()
pd.DataFrame(columns=cols, data=people) # [c[0] for c in cursor.description], 

,PersonID,PersonName,RoleID,CurrentBalance
0,1,Mike,2,0
1,2,Michelle,2,0


In [67]:
r = cursor.execute(f"""
    INSERT INTO chorerates (Rate, StartDate) VALUES (25, '2023-01-01 19:39:00');
""")
conn.commit()

In [68]:
import pandas as pd

In [70]:
rates = cursor.execute(""" select rate, datetime(StartDate) as StartDate from chorerates """).fetchall()
pd.DataFrame(rates, columns=[c[0] for c in cursor.description])

,Rate,StartDate
0,25,2023-01-01 00:00:00
1,25,2023-01-01 19:39:00
